In [ ]:
import pandas as pd
from os import remove

from classes import dbAzure
DBAzure = dbAzure.DBAzure

from helpers import correoEnvioArchivo, validarProduccion, rutas, correosDestino
estadoProduccion    = validarProduccion.validarProduccion()
rutaRelativa        = rutas.rutas()

from helpers import rutas, correoEnvioArchivo, correosDestino, validarProduccion
correoEnvioArchivo  = correoEnvioArchivo.correoEnvioArchivo

In [ ]:
rutaRelativa = rutaRelativa.replace('\\SRC', '')
rutaRelativa

In [ ]:
dbAzure = DBAzure(estadoProduccion)

In [ ]:
equipos = dbAzure.getQuerySolpeds()
#solpeds
display(equipos)

In [ ]:
equipos_envio = equipos.drop(['temperaturaMinima','temperaturaMaxima','Creado', 'DESCRIPCION', 'Modificado','created_at', 'updated_at', 'id' ], axis=1)
display(equipos)

In [ ]:
rutaRelativa.replace('/PROYECTOS', '/PROYECTOS/ROTACION/DOCS')

In [ ]:
equipos_envio.to_excel("EQUIPOS_NO_OPERATIVOS.xlsx")
#remove("EQUIPOS_NO_OPERATIVOS.xlsx")


In [ ]:
rutaRelativa = rutaRelativa.replace('/PROYECTOS', '/PROYECTOS/ROTACION')
rutaRelativa


In [ ]:
correos         = correosDestino.correosDestino(estadoProduccion)
correos['to'].append('camilomansillaulloa@gmail.com')
correos

In [ ]:
archivos    = []
archivos.append({
    'nombre_archivo':   'EQUIPOS_NO_OPERATIVOS.xlsx',
    'tipo_archivo':     'application',
    'formato_archivo': 'vnd.ms-excel',
    'ruta':             f'{rutaRelativa}/EQUIPOS_NO_OPERATIVOS.xlsx'
})
correoEnvioArchivo('EQUIPOS NO OPERATIVOS DS', "", correos, archivos)

In [ ]:
equipos['ESTADO_CORREO'] = 'NO ENVIAR'
display(equipos)

In [ ]:
for index in equipos.index:
    dbAzure.changeStatusSolped(equipos.iloc[index])

In [ ]:
#fileHtml    = open(f'{rutaRelativa}/templates/CorreoGerenteDS.html', 'r', encoding='utf-8')
#htmlBase    = fileHtml.read()
#fileHtml.close()
#htmlBase

In [ ]:
#fileHtml    = open(f'{rutaRelativa}/templates/correoAprobacionGerenteBanderas.html', 'r', encoding='utf-8')
#htmlBase    = fileHtml.read()
#fileHtml.close()

In [ ]:
for bandera in banderas:
    correos         = correosDestino.correosDestino(estadoProduccion)
    dataBandera     = solpeds[solpeds['bandera_estandarizada'] == bandera]
    if dataBandera.shape[0] == 0:
        continue
    html            = htmlBase
    tabla           = dataBandera[[
    'solped',
    'bandera',
    'centro',
    'nombre_local',
    'familia',
    'descripcion',
    'monto',
    'fecha_creacion_solped',
    ]].to_html(index=False)
    tabla = tabla.replace('\\n','<br>')
    tabla = tabla.replace('\n','')
    tabla = tabla.replace(' border="1" class="dataframe"','')
    tabla = tabla.replace(' style="text-align: right;"','')
    
    html = html.replace('{{TABLA}}', tabla)
    html = html.replace('{{ROWS}}', str(dataBandera.shape[0]))
    html = html.replace('{{NAME}}', banderas[bandera]['name'])
    html
    if estadoProduccion:
        correos['to'].append(banderas[bandera]['email'])
    correoEnvioArchivo(
    #    'Solped para ok',
    #    html,
    #    correos,
    #    [],
    #)
    
    dataBandera['estado'] = 'PENDIENTE APROBACION GERENTE DE BANDERA'
    dataBandera = dataBandera.reset_index(drop=True)
    display(dataBandera)
    
    for index in dataBandera.index:
        #print(index)
        dbAzure.changeStatusSolped(dataBandera.iloc[index])
        dbAzure.createHistory(dataBandera.iloc[index])

In [ ]:
solpeds = dbAzure.getQuerySolpeds()
solpeds